#Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!nvidia-smi

Fri May 15 20:11:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#Imports

In [3]:
import os
import json
import imageio
import csv
import numpy as np
from operator import itemgetter

from keras.utils import to_categorical
from keras.models import load_model

import matplotlib.pyplot as plt
import matplotlib

from scipy.stats import mode
import time

import sys
sys.path.append("/content/drive/My Drive/tf-keras-SegNet")
from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D

Using TensorFlow backend.


In [0]:
base_dir = "/content/drive/My Drive"

#Configurations and Parameters


In [0]:
def create_session_paths(session_name, overwrite=False, base_dir='/content/drive/My Drive/'):
    models_dir = os.path.join(base_dir, "models")
    session_dir = os.path.join(models_dir, session_name)
    # Prevent accidental overwriting of previous sessions
    try:
        os.mkdir(session_dir)
    except FileExistsError:
        if not overwrite:
            print("Set overwrite to True if you wish to continue")
            raise FileExistsError
        print("overwriting session")

    model = os.path.join(session_dir, "model.h5")
    history = os.path.join(session_dir, "history.json")
    training_log = os.path.join(session_dir, "logs.csv")
    training_config = os.path.join(session_dir, "config.json")
    classification_report = os.path.join(session_dir, "classification_report.txt")
    return {"model": model,
            "history": history,
            "logs": training_log,
            "config": training_config,
            "classification_report": classification_report}

In [6]:
session_name = "one_percent_transfer"
session_paths = create_session_paths(session_name, overwrite=True)

overwriting session


In [0]:
def get_image_list(metadata_file_path):
    with open(metadata_file_path, 'r') as f:
        return [i for i in csv.reader(f) if i]

In [0]:
model_dir = os.path.join(base_dir, "models")

qc_dir = os.path.join(base_dir, "qc")
qc_chunk_dir = os.path.join(qc_dir, "manual_labels")
window_path = os.path.join(qc_dir, "chunk_windows.json")

metadata_dir = os.path.join(base_dir, "Metadata")
training_set_path=os.path.join(metadata_dir, 'train.txt')
val_set_path=os.path.join(metadata_dir, 'val.txt')
test_set_path=os.path.join(metadata_dir, 'test.txt')

model_dir = os.path.join(base_dir, "models")
model_path=os.path.join(model_dir, 'one_percent_transfer')

best_model = "model.h5"

results_dir = os.path.join(base_dir, "results", best_model)

test_set_path = "/content/drive/My Drive/Metadata/manual_chunks.csv"

test_set_list = get_image_list(test_set_path)

bands = [2,3,4]
num_classes = 2
image_shape = (512, 512, len(bands))
padding = ((0, 0), (0, 0))
batch_size = 5 
          
test_set_size = len(test_set_list)

In [0]:
def data_gen(metadata_file_path, bands, batch_size):
    image_list = np.asarray(get_image_list(metadata_file_path))
    np.random.seed(1)
    np.random.shuffle(image_list)

    band_normalization_map = []
    counter = 0

    total_steps = image_list.shape[0] // batch_size
    while True:
        step_start = counter * batch_size
        step_end = step_start + batch_size
        images = []
        masks = []
        for j in range(step_start, step_end):
            images.append(np.load(image_list[j, 0])[:,:,bands])
            masks.append(np.load(image_list[j, 1]))

        y = to_categorical(np.array(masks))
        yield np.array(images), y.reshape((batch_size, y.shape[1] * y.shape[2], y.shape[3]))

        counter +=1

        if counter >= total_steps:
            counter = 0

#Predictor


In [33]:
tic = time.time()
print("start time: {}".format(tic))
model_name = best_model
current_model = os.path.join(model_path, model_name)
assert os.path.exists(current_model)
toc = time.time()

test_data = data_gen(test_set_path, bands, batch_size)

toc = time.time() - toc
print("test_data loaded in {0:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))

my_model = load_model(session_paths["model"], 
                      custom_objects={"MaxPoolingWithArgmax2D": MaxPoolingWithArgmax2D, 
                                      "MaxUnpooling2D": MaxUnpooling2D
                                      }
                      )
toc = time.time() - toc
print("model created in {0:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))

probs = my_model.predict(test_data, steps = test_set_size // batch_size, verbose=1)
toc = time.time() - toc
print("predictions generated in {0:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))

start time: 1589577208.0670705
test_data loaded in 0.00 seconds
total runtime: 0.00
model created in 1589577247.47 seconds
total runtime: 39.40
6/6 [==============================] - 10s 2s/step
predictions generated in 10.96 seconds
total runtime: 50.36


In [0]:
probs = probs.reshape((probs.shape[0], image_shape[0], image_shape[1], num_classes))

In [0]:
output = probs.argmax(axis=-1)

In [0]:
hist = np.histogram(output, bins=[0,1,2])

In [37]:
print(hist)

(array([7544550,  319770]), array([0, 1, 2]))


In [38]:
test_scene_dir = "/content/drive/My Drive/test_chunks"
output_dir = "/content/drive/My Drive/output"

if not os.path.exists(os.path.join(output_dir, best_model)):
  os.mkdir(os.path.join(output_dir, best_model))

test_scene_ids = [i for i in os.listdir(test_scene_dir) if os.path.isdir(os.path.join(test_scene_dir, i))]
print(test_scene_ids)
for i in test_scene_ids:
  output_scene_dir = os.path.join(output_dir, best_model, i)
  if not os.path.exists(output_scene_dir):
    os.mkdir(output_scene_dir)
    print("output_dir made at {}".format(output_scene_dir))

['LC82201072015017LGN00', 'LC82071182013336LGN00', 'LC81431082013352LGN00', 'LC81681112014002LGN00', 'LC80631112014002LGN00', 'LC80311222014338LGN00', 'LC81041072013303LGN00', 'LC82091172014001LGN00', 'LC82131132013362LGN00']


In [39]:
for i in range(output.shape[0]):
    chunk_path, chunk_name = os.path.split(test_set_list[i][0])
    scene_chunk_path, scene_id = os.path.split(chunk_path)
    name, ext = os.path.splitext(chunk_name)
    out_file_name = f"{name}_output{ext}"
    print(os.path.join(output_dir, model_name, scene_id, out_file_name))
    np.save(os.path.join(output_dir, model_name, scene_id, out_file_name), output[i])

/content/drive/My Drive/output/model.h5/LC80311222014338LGN00/chunk_6_1_output.npy
/content/drive/My Drive/output/model.h5/LC80311222014338LGN00/chunk_6_2_output.npy
/content/drive/My Drive/output/model.h5/LC80311222014338LGN00/chunk_7_1_output.npy
/content/drive/My Drive/output/model.h5/LC80311222014338LGN00/chunk_7_2_output.npy
/content/drive/My Drive/output/model.h5/LC80631112014002LGN00/chunk_6_6_output.npy
/content/drive/My Drive/output/model.h5/LC80631112014002LGN00/chunk_6_7_output.npy
/content/drive/My Drive/output/model.h5/LC81041072013303LGN00/chunk_6_14_output.npy
/content/drive/My Drive/output/model.h5/LC81041072013303LGN00/chunk_7_14_output.npy
/content/drive/My Drive/output/model.h5/LC81431082013352LGN00/chunk_3_6_output.npy
/content/drive/My Drive/output/model.h5/LC81431082013352LGN00/chunk_4_6_output.npy
/content/drive/My Drive/output/model.h5/LC81681112014002LGN00/chunk_6_2_output.npy
/content/drive/My Drive/output/model.h5/LC81681112014002LGN00/chunk_6_3_output.npy
/c

#Ensemble Predictor

In [0]:
# import argparse
# from keras.models import Model
# from keras.layers import Activation,Input
# from scipy.stats import mode
# import os
# #from custom_layers import MaxPoolingWithIndices,UpSamplingWithIndices,CompositeConv
# #import config as cf
# #from util import read_image_batch
# #from create_model import create_model

# class_labels=[]


# for i in os.listdir(model_path):
#   if(i==".ipynb_checkpoints"):
#     continue
#   print(model_path+i)
#   my_model=create_model()
#   my_model.compile(optimizer,loss=loss_function,metrics=metrics)
#   my_model.load_weights(model_path+i)
#   key=int(i[-1:])
#   if(key!=0):
#     continue
#   test_data=read_test_image_batch(test_set_path,batch_size,set_info[key])
  
#   probs=my_model.predict(test_data,steps=(test_set_size+1)//batch_size)
  
#   class_labels.append(probs.argmax(axis=-1))
  
  
# class_labels=np.asarray(class_labels)
# print(class_labels.shape)



# u,indices=np.unique(class_labels,return_inverse=True)
# final_labels=u[np.argmax(np.apply_along_axis(np.bincount,0,indices.reshape(class_labels.shape),None,np.max(indices)+1),axis=0)]

# print("final labels")
# print(final_labels.shape)
# #print("\n\n")
# #print(final_labels)


# for i in range(test_set_size):
#   line=test_names[i]
#   print(line)
#   str_array=line.split(" ")
#   scene_id=str_array[1][18:40]
#   out_str=str_array[1][40:-4]+"_output.npy"
#   print(out_str)
#   np.save('/content/drive/My Drive/new_test_outputs_key_1/'+scene_id+out_str,final_labels[i])
  


# Metric Generation

## Define Functions

In [0]:
def diff_stats(diff_hist):
    stats = {}
    stats["total_pixels"] = diff_hist[0] + diff_hist[1] + diff_hist[2] + diff_hist[3]
    stats["correct"] = diff_hist[0] + diff_hist[3]
    stats["incorrect"] = diff_hist[1] + diff_hist[2]
    
    stats["notrock"] = diff_hist[0]
    stats["false_notrock"] = diff_hist[1]
    stats["false_rock"] = diff_hist[2]
    stats["rock"] = diff_hist[3]
                    
    stats["accuracy"] = stats["correct"] / stats["total_pixels"]
    stats["classification_accuracy"] = stats["rock"] / (stats["rock"] + stats["false_rock"] + stats["false_notrock"])
    stats["rock_omission"] = diff_hist[1] / (diff_hist[1] + diff_hist[3])
    stats["rock_commission"] = diff_hist[2] / (diff_hist[2] + diff_hist[3])
    stats["notrock_omission"] = diff_hist[2] / (diff_hist[0] + diff_hist[2])
    stats["notrock_commission"] = diff_hist[1] / (diff_hist[0] + diff_hist[1])
    stats["f1_rock"] = 2 * ((stats["rock_omission"] * stats["rock_commission"]) / (stats["rock_omission"] + stats["rock_commission"]))

    stats["rock_producers"] = diff_hist[3] / (diff_hist[1] + diff_hist[3]) * 100
    stats["notrock_producers"] = diff_hist[0] / (diff_hist[0] + diff_hist[2]) * 100
    stats["rock_users"] = diff_hist[3] / (diff_hist[2] + diff_hist[3]) * 100
    stats["notrock_users"] = diff_hist[0] / (diff_hist[0] + diff_hist[1]) * 100

    return stats

In [0]:
def assemble_scene_output(scene_output_dir, window, suffix):
    chunks = []
    row_vals = [i[0] for i in window['chunks']]
    col_vals = [i[1] for i in window['chunks']]
    row_range = (min(row_vals), max(row_vals))
    col_range = (min(col_vals), max(col_vals))
   
    mosaic_rows = []
    for i in range(row_range[0], row_range[1] + 1):
        row = []
        for j in range(col_range[0], col_range[1] + 1):
            row.append(np.load(os.path.join(scene_output_dir, "chunk_{}_{}_{}.npy".format(i, j, suffix))))
        
        mosaic_rows.append(np.concatenate(tuple(row), axis=1))
    
    mosaic = np.concatenate(tuple(mosaic_rows), axis=0)
    
    return mosaic[window["row"]: window["row"] + window["height"],
                            window["col"]: window["col"] + window["width"]]

## Define Data Paths

In [0]:
model_name = best_model
model_path = os.path.join(model_dir, model_name)
output_path = os.path.join(base_dir, "output", model_name)
special_window = "ryder_bay_abj"

## Load Window Metadata

In [0]:
with open(window_path, 'r') as window_file:
    windows = json.load(window_file)

In [0]:
print(windows)

{'LC80311222014338LGN00': {'chunks': [[6, 1], [6, 2], [7, 1], [7, 2]], 'row': 317, 'col': 498, 'width': 333, 'height': 333}, 'LC80631112014002LGN00': {'chunks': [[6, 6], [6, 7]], 'row': 165, 'col': 272, 'width': 333, 'height': 333}, 'LC81041072013303LGN00': {'chunks': [[6, 14], [7, 14]], 'row': 282, 'col': 20, 'width': 333, 'height': 334}, 'LC81431082013352LGN00': {'chunks': [[3, 6], [4, 6]], 'row': 508, 'col': 64, 'width': 333, 'height': 334}, 'LC81681112014002LGN00': {'chunks': [[6, 2], [6, 3], [7, 2], [7, 3]], 'row': 391, 'col': 318, 'width': 334, 'height': 334}, 'LC82071182013336LGN00': {'chunks': [[7, 8], [7, 9], [8, 8], [8, 9]], 'row': 488, 'col': 482, 'width': 334, 'height': 334}, 'LC82091172014001LGN00': {'chunks': [[15, 6], [15, 7], [16, 6], [16, 7]], 'row': 296, 'col': 505, 'width': 333, 'height': 333}, 'LC82131132013362LGN00': {'chunks': [[14, 9], [15, 9]], 'row': 456, 'col': 160, 'width': 333, 'height': 333}, 'LC82201072015017LGN00': {'chunks': [[8, 13], [8, 14]], 'row': 15

## Load Output and manual chunks

In [0]:
output_chunks = {}
label_chunks = {}
man_chunks = {}

for i in windows:
    if i == "ryder_bay_abj":
        scene_dir = os.path.join(output_path, "LC82201072015017LGN00")
        label_dir = os.path.join(base_dir, "test_chunks", "LC82201072015017LGN00")
    else:
        scene_dir = os.path.join(output_path, i)
        label_dir = os.path.join(base_dir, "test_chunks", i)
    if os.path.exists(scene_dir):
        output_chunks[i] = assemble_scene_output(scene_dir, windows[i], "output")
        man_path = os.path.join(qc_chunk_dir, i + "_man.npy")
        man_chunks[i] = np.load(man_path)
        label_chunks[i] = assemble_scene_output(label_dir, windows[i], "label")

# Analyze Outputs

## Generate Diff Rasters

In [0]:
diffs = {}
for i in output_chunks:
    actual = output_chunks[i] * 2 
    diffs[i] = man_chunks[i] + actual

In [0]:
label_diffs = {}
for i in label_chunks:
    actual = label_chunks[i] * 2 
    label_diffs[i] = man_chunks[i] + actual

## Generate Diff Histograms

In [0]:
diff_hists = {}
for i in diffs:
    diff_hists[i] = np.histogram(diffs[i], bins=[0,1,2,3,4])[0]

In [0]:
label_hists = {}
for i in label_diffs:
    label_hists[i] = np.histogram(label_diffs[i], bins=[0,1,2,3,4])[0]

In [0]:
print(diff_hists)
print(label_hists)

{'LC80311222014338LGN00': array([100091,   4577,   5247,    974]), 'LC80631112014002LGN00': array([102763,   5958,    719,   1449]), 'LC81041072013303LGN00': array([95866,  3061, 11399,   896]), 'LC81431082013352LGN00': array([86574, 18212,  1633,  4803]), 'LC81681112014002LGN00': array([100277,   8411,   1660,   1208]), 'LC82071182013336LGN00': array([102436,   7945,    668,    507]), 'LC82091172014001LGN00': array([108786,    158,   1929,     16]), 'LC82131132013362LGN00': array([107776,   1977,    683,    453]), 'LC82201072015017LGN00': array([95387,  8489,  3917,  3096]), 'ryder_bay_abj': array([93023,  2250, 15251,   365])}
{'LC80311222014338LGN00': array([104692,   1040,    646,   4511]), 'LC80631112014002LGN00': array([102872,    868,    610,   6539]), 'LC81041072013303LGN00': array([105626,    112,   1639,   3845]), 'LC81431082013352LGN00': array([86269,   721,  1938, 22294]), 'LC81681112014002LGN00': array([101126,   1732,    811,   7887]), 'LC82071182013336LGN00': array([1012

## Calculate Metrics

In [0]:
diff_statistics = {}
for i in diff_hists:
    diff_statistics[i] = diff_stats(diff_hists[i])

In [0]:
label_statistics = {}
for i in label_hists:
    label_statistics[i] = diff_stats(label_hists[i])

## Print Metrics

In [0]:
for i in diff_statistics:
    if i != "aggregate":
        print("{0}\nOverall accuracy: {1:.2f}\nclassifcation accuracy: {2:.2f}\n".format(i, diff_statistics[i]["accuracy"], diff_statistics[i]["classification_accuracy"]))

LC80311222014338LGN00
Overall accuracy: 0.91
classifcation accuracy: 0.09

LC80631112014002LGN00
Overall accuracy: 0.94
classifcation accuracy: 0.18

LC81041072013303LGN00
Overall accuracy: 0.87
classifcation accuracy: 0.06

LC81431082013352LGN00
Overall accuracy: 0.82
classifcation accuracy: 0.19

LC81681112014002LGN00
Overall accuracy: 0.91
classifcation accuracy: 0.11

LC82071182013336LGN00
Overall accuracy: 0.92
classifcation accuracy: 0.06

LC82091172014001LGN00
Overall accuracy: 0.98
classifcation accuracy: 0.01

LC82131132013362LGN00
Overall accuracy: 0.98
classifcation accuracy: 0.15

LC82201072015017LGN00
Overall accuracy: 0.89
classifcation accuracy: 0.20

ryder_bay_abj
Overall accuracy: 0.84
classifcation accuracy: 0.02



In [0]:
for i in label_statistics:
    print("{}\nOverall accuracy: {}\nclassifcation accuracy: {}\n".format(i, label_statistics[i]["accuracy"], label_statistics[i]["classification_accuracy"]))

LC80311222014338LGN00
Overall accuracy: 0.9847956064172281
classifcation accuracy: 0.7279328707439083

LC80631112014002LGN00
Overall accuracy: 0.9866713560407254
classifcation accuracy: 0.8156417612573281

LC81041072013303LGN00
Overall accuracy: 0.984256711801622
classifcation accuracy: 0.6870979270907791

LC81431082013352LGN00
Overall accuracy: 0.9760928593263922
classifcation accuracy: 0.8934396665731575

LC81681112014002LGN00
Overall accuracy: 0.9772042740865574
classifcation accuracy: 0.7561840843720038

LC82071182013336LGN00
Overall accuracy: 0.9597511563698949
classifcation accuracy: 0.5661416561986665

LC82091172014001LGN00
Overall accuracy: 0.9990801612423234
classifcation accuracy: 0.5767634854771784

LC82131132013362LGN00
Overall accuracy: 0.9953106259412565
classifcation accuracy: 0.8009188361408882

LC82201072015017LGN00
Overall accuracy: 0.9646763880998115
classifcation accuracy: 0.6944141051646122

ryder_bay_abj
Overall accuracy: 0.9930110290470651
classifcation accuracy:

In [0]:
import statistics
from statistics import StatisticsError
def aggregate_stats(diff_statistics):
    diff_statistics.pop("aggregate", None)    
    result = {}
    aggregate = list(diff_statistics.values())
    for metric in aggregate[0].keys(): 
        try:
            result[metric] = {}
            result[metric]["mean"] = statistics.mean([i[metric] for i in aggregate if not isinstance(i[metric], (str, dict))])
            result[metric]["stdev"] = statistics.stdev([i[metric] for i in aggregate if not isinstance(i[metric], (str,dict))])
        except StatisticsError as e:
            print(e)
    return result



In [0]:
diff_statistics["aggregate"] = aggregate_stats(diff_statistics)
label_statistics["aggregate"] = aggregate_stats(label_statistics)

# Save Results

In [0]:
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

def convert_int(val):
    if isinstance(val, np.int64): return int(val)  
    raise TypeError

with open(os.path.join(results_dir, "{}.JSON".format(best_model)), 'w+') as out:
    json.dump(diff_statistics, out, default=convert_int)

# Print Results

In [0]:
with open(os.path.join(results_dir, "{}.JSON".format(best_model))) as res:
    stats = json.load(res)

In [0]:
print(results_dir, best_model)

/content/drive/My Drive/results/model_Bands_All_NoClouds_focal_loss_epoch20_lr_0.01 model_Bands_All_NoClouds_focal_loss_epoch20_lr_0.01


In [0]:
import pprint

In [0]:
pprint.pprint(stats)

{'LC80311222014338LGN00': {'accuracy': 0.9114069023978933,
                           'classification_accuracy': 0.09020188923874792,
                           'correct': 101065,
                           'f1_rock': 0.8338777756392604,
                           'false_notrock': 4577,
                           'false_rock': 5247,
                           'incorrect': 9824,
                           'notrock': 100091,
                           'notrock_commission': 0.04372874230901517,
                           'notrock_omission': 0.049811084319049154,
                           'notrock_producers': 95.01889156809509,
                           'notrock_users': 95.62712576909848,
                           'rock': 974,
                           'rock_commission': 0.8434335315865616,
                           'rock_omission': 0.8245361196180868,
                           'rock_producers': 17.546388038191317,
                           'rock_users': 15.656646841343836,
        